In [ ]:
import dill
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import plotly.express as px
import plotly.graph_objs as go
print('done')
import csv


USE_PLOTLY = True


In [ ]:
def load_from_csv(input_string):
    with open(input_string, 'r') as fp:
        reader = csv.reader(fp)
        li = list(reader)
    newlist = []
    for row in li:
        newrow = []
        for entry in row [1:]:
            newrow.append(float(entry))
        newlist.append(newrow)
    return newlist


experiment_config_results = load_from_csv("experiment_config_results_gpu.csv")

with open("experiments_gpu","rb") as current_file:
    experiments = dill.load(current_file)

    
non_unique_params = np.array([experiment['parameter'] for experiment in experiments])
non_unique_config_strings = np.array([experiment['config_string'] for experiment in experiments])

par_indices = np.unique(non_unique_params,return_index=True)[1]
string_indices = np.unique(non_unique_config_strings,return_index=True)[1]

parameters = non_unique_params[np.sort(par_indices)]
config_strings = non_unique_config_strings[np.sort(string_indices)]
# config_strings[0],config_strings[1],config_strings[2] = config_strings[2],config_strings[0],config_strings[1]
# config_strings[[0,1,2]] = config_strings[[2,0,1]]
print(config_strings)

def str_noneguard(obj):
    if hasattr(obj,'__name__'):
        return obj.__name__
    if obj is None:
        return ''
    if isinstance(obj,list):
        return str([str_noneguard(x) for x in obj])
    return str(obj)


print(len(config_strings))
# colors=['#4285F4','#EA4335', '#FBBC04',  '#34A853','#000000','#46BDC6',      '#46BDC6',             '#F4CCCC','#C9DAF8',            'grey','purple']
colors=['#EA4335', '#FBBC04', '#4285F4', '#000000','#34A853','#46BDC6',      '#46BDC6',             '#F4CCCC','#C9DAF8',            'grey','purple']
print(len(colors))
print(config_strings)
print("")
print("")

print(parameters)
print("")

# parameters[[3,4,5,6,7,8,9,10,11,12]] = parameters[[-4,-3,-2,5,3,4,6,8,-1,7]]
print(parameters)



In [ ]:
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

def hex_to_rgb(hex_color: str) -> tuple:
    # thanks to https://community.plotly.com/t/scatter-plot-fill-with-color-how-to-set-opacity-of-fill/29591/2
    hex_color = hex_color.lstrip("#")
    if len(hex_color) == 3:
        hex_color = hex_color * 2
    return int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)

def plot_vals(i,ylim=[-175,100]):

    if USE_PLOTLY:
        fig = go.Figure()

    else:
        fig, ax = plt.subplots(figsize=[10,6])  
        ax.set_title(parameter)

    parameter = parameters[i]
    bar=0
    all_numbers = True
    
    for config_index,config_string in enumerate(config_strings):
        color=colors[config_index]
    #     print("--------- " + config_string + " -----------")
        filtered_experiments = [ex for ex in experiments if ex['parameter']==parameter and ex['config_string']==config_string]
        if len(filtered_experiments)==0:
            continue
        mappings = np.array([x['mapping'] for x in filtered_experiments])
        x_values = [x['current_var'] for x in filtered_experiments]
        for x in x_values:
            if not isinstance(x,int) and not isinstance(x,float):
                all_numbers=False

        if not all_numbers:
            for i in range(len(x_values)):
                x_values[i] = str_noneguard(x_values[i])

        results = [result for result in np.array(experiment_config_results,dtype=object)[mappings] ]
        means = np.array([np.mean(x) for x in results])
        sigmas = np.array([np.std(x) for x in results])

    #     print("mappings")
    #     display(mappings)
    #     print("")
    #     print("results")
    #     display(results)
    #     print("")
    #     print("x_values")
    #     display(x_values)
    #     print("")
    #     print("means")
    #     display(means)
    #     print("")
    #     print("sigmas")
    #     display(sigmas)
    #     print("\n\n\n")
    #     ax.plot(x_values,means)
#         print(x_values)
        if USE_PLOTLY:
            if all_numbers:
                # fig.add_trace(go.Scatter(x=x_values, y=means, error_y={"array":sigmas}, marker_color=color,name=config_string,mode='lines+markers'))
                fig.add_trace(go.Scatter(x=x_values, y=means, marker_color=color,name=config_string,mode='lines+markers'))
                fig.add_trace(go.Scatter(x=x_values, y=means+sigmas, marker_color=color,name="Upper bound", line=dict(width=0), showlegend=False, mode='lines'))
                fig.add_trace(go.Scatter(x=x_values, y=means-sigmas, marker_color=color,fillcolor=f"rgba{(*hex_to_rgb(color), 0.2)}",name="Lower bound", line=dict(width=0), showlegend=False, fill='tonexty', mode='lines'))
                
            else:
                fig.add_bar(x=x_values,y=means,error_y={"array":sigmas},marker_color=color,name=config_string)
        else:
            if all_numbers:
                ax.plot(x_values,means,'o-',label=config_string,color=color)
                ax.fill_between(x_values, (means-sigmas), (means+sigmas),alpha=.2,color=color)
            else:
                n=len(x_values)
                ind = np.arange(n)
                width = 0.1
                position = ind + (width*(1-n)/2) + bar*width
                ax.bar(position,means,width=width,yerr=sigmas,label=config_string,color=color)
                bar+=1
    
    if USE_PLOTLY:
        fig.update_yaxes(tick0=0, dtick=25)
        # fig.update_xaxes(tickangle = 0)
        fig.update_layout(yaxis_range=ylim,title=parameter, width=1000, height=600)
        fig.show()
    else:
        ax.set_ylim(ylim)
        if not all_numbers:
            print(x_values)
            print(ind)
    #         ax.axhline(0,"black")
            ax.yaxis.set_major_locator(MultipleLocator(25))    
            ax.grid(axis="y")
            
            plt.xticks(ind, x_values)
        else:
            ax.yaxis.set_major_locator(MultipleLocator(25))
            ax.grid()
        ax.axhline(0,color='black')
        ax.legend()

In [ ]:
plot_vals(0)
plt.savefig("pictures/results_fig_0.svg")

In [ ]:
plot_vals(1)
plt.savefig("pictures/results_fig_1.svg")

In [ ]:
plot_vals(2)
plt.savefig("pictures/results_fig_2.svg")

In [ ]:
plot_vals(3)
plt.savefig("pictures/results_fig_3.svg")

In [ ]:
plot_vals(4)
plt.savefig("pictures/results_fig_4.svg")

In [ ]:
plot_vals(5)
plt.savefig("pictures/results_fig_5.svg")

In [ ]:
plot_vals(6)
plt.savefig("pictures/results_fig_6.svg")

In [ ]:
plot_vals(7)
plt.savefig("pictures/results_fig_7.svg")

In [ ]:
plot_vals(8)
plt.savefig("pictures/results_fig_8.svg")

In [ ]:
plot_vals(9)
plt.savefig("pictures/results_fig_9.svg")

In [ ]:
plot_vals(10)
plt.savefig("pictures/results_fig_10.svg")

In [ ]:
# plot_vals(11)
# plt.savefig("pictures/results_fig_11.svg")

In [ ]:
# plot_vals(12,ylim=[-350,100])
# plt.savefig("pictures/results_fig_12.svg")

In [ ]:
# a=[len(experiment_config_results[x['mapping']]) for i,x in enumerate(experiments)]
# unique, counts = np.unique(a, return_counts=True)
# dict(zip(unique, counts))


[str(i) + " " + str(len(experiment_config_results[x['mapping']])) for i,x in enumerate(experiments)]

In [ ]:
suffix1 = ""
suffix2 = "_gpu"
result = "_merge1"

In [ ]:
experiment_config_results1 = load_from_csv("experiment_config_results"+suffix1+".csv")
experiment_config_results2 = load_from_csv("experiment_config_results"+suffix2+".csv")

experiment_config_results = []
for i in range(len(experiment_config_results1)):
    exp1 = experiment_config_results1[i]
    exp2 = experiment_config_results2[i]
    experiment_config_results.append(exp1+exp2)

display(experiment_config_results)

experiment_config_strings = []
experiment_config_list = []
for ex in experiments:
    if ex['config'] not in experiment_config_list:
        experiment_config_list.append(ex['config'])
        experiment_config_strings.append(ex['full_string'])


experiment_configs_csv = [[experiment_config_strings[i]] + experiment_config_results[i] for i in range(len(experiment_config_results))]

with open("experiment_config_results"+result+".csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(experiment_configs_csv)


In [ ]:
experiment_config_results1 = dill.load("experiment_config_results2"+suffix1+".csv")
experiment_config_results2 = dill.load("experiment_config_results2"+suffix2+".csv")

experiment_config_results = []
for i in range(len(experiment_config_results1)):
    exp1 = experiment_config_results1[i]
    exp2 = experiment_config_results2[i]
    experiment_config_results.append(exp1+exp2)

display(experiment_config_results)

experiment_config_strings = []
experiment_config_list = []
for ex in experiments:
    if ex['config'] not in experiment_config_list:
        experiment_config_list.append(ex['config'])
        experiment_config_strings.append(ex['full_string'])


experiment_configs_csv = [[experiment_config_strings[i]] + experiment_config_results[i] for i in range(len(experiment_config_results))]

with open("experiment_config_results2"+result+".csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(experiment_configs_csv)

In [ ]:
# dill to csv

# import csv
# import dill

# with open("experiment_config_results_gpu","rb") as current_file:
#     experiment_config_results = dill.load(current_file)
# with open("experiment_config_results2_gpu","rb") as current_file:
#     experiment_config_results2 = dill.load(current_file)    
# with open("experiments_gpu","rb") as current_file:
#     experiments = dill.load(current_file)

# experiment_config_strings = []
# experiment_config_list = []
# for ex in experiments:
#     if ex['config'] not in experiment_config_list:
#         experiment_config_list.append(ex['config'])
#         experiment_config_strings.append(ex['full_string'])


# experiment_configs_csv = [[experiment_config_strings[i]] + experiment_config_results[i] for i in range(len(experiment_config_results))]
# experiment_configs2_csv = [[experiment_config_strings[i]] + experiment_config_results2[i] for i in range(len(experiment_config_results))]

# with open("experiment_config_results_gpu.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(experiment_configs_csv)
# with open("experiment_config_results2_gpu.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(experiment_configs2_csv)  

In [ ]:
# csv to dict

# import csv
# with open('experiment_config_results_gpu.csv', 'r') as fp:
#     reader = csv.reader(fp)
#     li = list(reader)
# newlist = []
# for row in li:
#     newrow = []
#     for entry in row [1:]:
#         newrow.append(float(entry))
#     newlist.append(newrow)
